# Training Protein Embeddings

In this notebook, we will preprocess the data that will be used for training a protein function prediction model, i.e. we will embed the training protein sequences into a vector format.

## Libraries

In [ ]:
!pip install Bio

In [ ]:
import re
import torch
import pandas as pd
import numpy as np

from Bio import SeqIO
from tqdm import tqdm
from transformers import T5Tokenizer, T5EncoderModel

## Data

The objective of the model will be to predict the terms (functions) of a protein sequence. It's important to keep in mind that one protein sequence can have many functions (GO Term IDs) and all of them must be predicted by our model for each protein sequence.

### Protein Sequence

Our data is composed of protein sequences (a string of letters), where each one-letter or three-letter code represents an amino acid. The sequences can be found in the file `train_sequences.fasta`.

In [ ]:
proteins = SeqIO.parse('./data/cafa-5-protein-function-prediction/Train/train_sequences.fasta', "fasta")
train_proteins = {}

for protein in proteins:
    train_proteins[protein.id] = {'sequence': str(protein.seq), 'GO': {'BPO': [], 'CCO': [], 'MFO': []}}

In [ ]:
list(train_proteins.items())[:3]

### Taxonomy

The file `train_taxonomy.tsv` contains list of proteins and the species to whuch they belong (taxonomy ID). The first columns is the protein UniProt accession ID and the second is the taxon ID.

In [ ]:
train_taxonomy = pd.read_csv('./data/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv', sep='\t')

In [ ]:
train_taxonomy.head(3)

### Gene Ontology

The functional properties of proteins are defined using Gene Ontology (GO) with respect to Molecular Function (MF), Biological Process (BP), and Cellular Component (CC). List of annotated terms (ground thruths) for the protein sequences are available in the file `train_terms.fasta`, where the first column (attribute) is the protein's UniProt accession ID (unique protein id), the second is the GO Term ID, and the third is the ontology, in which the term appears.

In [ ]:
train_terms = pd.read_csv('./data/cafa-5-protein-function-prediction/Train/train_terms.tsv', sep='\t')

In [ ]:
train_terms.head(3)

## Embedding

We will start by initializing the tokenizer and the model, which we will use to generate the protein embeddings. We will be using an encoder-only, half-precision version of the [ProtT5-XL-UniRef50](https://huggingface.co/Rostlab/prot_t5_xl_uniref50) model, which is pretrained on a large corpus of protein sequences in a self-supervised fashion. [This version](https://huggingface.co/Rostlab/prot_t5_xl_half_uniref50-enc) will help us generate protein embeddings even with low GPU-memort, because it is fully usable on 8 GB of video RAM.

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc")
model = model.to(device)
model = model.eval()
tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False)

In [ ]:
all_ids = list(train_proteins.keys())
sequences = sorted(
    [re.sub(r"[UZOB]", "X", protein['sequence']) for protein in train_proteins.values()],
    key=len,
    reverse=True
)

for seq in sequences[:3]:
    print(len(seq))

In [ ]:
batch = list()
max_batch=100
max_residues=4000
max_seq_len=1000

for idx, seq in enumerate(sequences):
    seq_len = len(seq)
    seq = ' '.join(list(seq))
    batch.append((all_ids[idx], seq, seq_len))

    n_res_batch = sum([s_len for _, _, s_len in batch]) + seq_len
    if len(batch) >= max_batch or n_res_batch >= max_residues or idx == len(sequences) or seq_len > max_seq_len:
        protein_ids, seqs, seq_lens = zip(*batch)
        batch = list()

        token_encoding = tokenizer.batch_encode_plus(seqs, add_special_tokens=True, padding="longest")
        input_ids = torch.tensor(token_encoding['input_ids']).to(device)
        attention_mask = torch.tensor(token_encoding['attention_mask']).to(device)

        try:
            with torch.no_grad():
                embedding_repr = model(input_ids=input_ids, attention_mask=attention_mask)
        except RuntimeError:
            print("RuntimeError during embedding for {} (L={})".format(protein_ids[idx], seq_len))
            continue

        for batch_idx, identifier in enumerate(protein_ids):
            s_len = seq_lens[batch_idx]
            protein_emb = embedding_repr.last_hidden_state[batch_idx,:s_len].mean(dim=0)

            train_proteins[identifier]['embedding'] = protein_emb.detach().cpu().numpy().squeeze()

In [ ]:
list(train_proteins.items())[:3]

## Integrating Data

Now when we have our protein sequences prepared, we can simply integrate the data, which will give us only one table, that contains all protein sequences and their corresponding taxonomy and GO Term IDs with respect to all aspects (MF, BP, and CC).

In [ ]:
for _, row in train_taxonomy.iterrows():
    if row['EntryID'] in train_proteins:
        train_proteins[row['EntryID']]['taxonomyID'] = row['taxonomyID']

In [ ]:
for _, row in train_terms.iterrows():
    if row['EntryID'] in train_proteins:
        train_proteins[row['EntryID']]['GO'][row['aspect']].append(row['term'])

In [ ]:
data_list = []
features = []
labels = []

for protein_id, data in train_proteins.items():
    data_list.append([
        protein_id,
        data['taxonomyID'],
        data['sequence'],
        data['embedding'],
        data['GO']['BPO'],
        data['GO']['CCO'],
        data['GO']['MFO']
    ])

    protein_features = [
        protein_id,
        data['taxonomyID'],
    ]

    protein_features.extend(data['embedding'])
    features.append(protein_features)

    labels.append([
        protein_id,
        data['GO']['BPO'],
        data['GO']['CCO'],
        data['GO']['MFO']
    ])

In [ ]:
train_df = pd.DataFrame(data_list, columns=['ProteinID', 'TaxonomyID', 'Sequence', 'Embedding', 'BPO', 'CCO', 'MFO'])
train_df.set_index('ProteinID', inplace=True)
train_df.index.name = None

In [ ]:
train_df.head(3)

In [ ]:
column_names = ['ProteinID', 'TaxonomyID'] + ['Embed_' + str(i+1) for i in range(1024)]
X_train = pd.DataFrame(features, columns=column_names)
X_train.set_index('ProteinID', inplace=True)
X_train.index.name = None

In [ ]:
X_train.head(3)

In [ ]:
y_train = pd.DataFrame(labels, columns=['ProteinID', 'BPO', 'CCO', 'MFO'])
y_train.set_index('ProteinID', inplace=True)
y_train.index.name = None

In [ ]:
y_train.head(3)

## Store

Let's save the embedded data as CSV files for both training so we could easily access them.

In [ ]:
train_df.to_csv('data_train.csv', index=False)

In [ ]:
X_train.to_csv('features_train.csv', index=False)

In [ ]:
y_train.to_csv('labels_train.csv', index=False)